#### 전이학습

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# 이미지 데이터셋, 전처리, 전이학습 모듈
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights

from torchinfo import summary
from torchmetrics.functional.classification import multiclass_accuracy 

import matplotlib.pyplot as plt

C:\ProgramData\anaconda3\envs\Torch_PY38\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 지정된 프로시저를 찾을 수 없습니다'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
# 데이터 준비
img_dir = '../data/img'

# Resnet 전처리
preprocessing = transforms.Compose([
    transforms.Resize(256, interpolation = transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
])

# 이미지 데이터셋 생성
imgDS = ImageFolder(root = img_dir, transform = preprocessing)

print(imgDS.classes, imgDS.targets, imgDS.imgs, sep='\n')

['folder1', 'folder2']
[0, 0, 0, 0, 0, 0, 0, 1]
[('../data/img\\folder1\\111.jpg', 0), ('../data/img\\folder1\\222.jpg', 0), ('../data/img\\folder1\\333.jpg', 0), ('../data/img\\folder1\\444.jpg', 0), ('../data/img\\folder1\\555.jpg', 0), ('../data/img\\folder1\\666.jpg', 0), ('../data/img\\folder1\\777.jpg', 0), ('../data/img\\folder2\\gray_3.jpg', 1)]


In [4]:
# 데이터 로더 생성
imgDL = DataLoader(imgDS, batch_size =3, shuffle = True, drop_last = True)
for (img, label) in imgDL:
    print(img.shape, label)

torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 1, 0])


In [5]:
### ===> 모델 설계 및 설정

# 사전학습된 모델 인스턴스 생성
res_model = resnet18(weights = ResNet18_Weights.DEFAULT)

# 전결합층 변경
# in_features : FeatureMap에서 받은 피처 수, out_features : 출력/분류 클래스수 
res_model.fc = nn.Linear(in_features = 512, out_features = 2)

# 이렇게 따로 떨어뜨려서 해도 되고 클래스에 다 담아서 돌려도 된다!

In [6]:
summary(res_model, input_size = (3, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [3, 2]                    --
├─Conv2d: 1-1                            [3, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [3, 64, 112, 112]         128
├─ReLU: 1-3                              [3, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [3, 64, 56, 56]           --
├─Sequential: 1-5                        [3, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [3, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [3, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [3, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [3, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [3, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [3, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [3, 64, 56, 56]           --
│

In [7]:
### Resnet18 Feature Module 파라미터 requires grad = True => False 비활성화
for name, param in res_model.named_parameters():
    print(name, param.requires_grad, end=' ====>    ')
    param.requires_grad = False
    print(param.requires_grad)

print('=================================================================')
### Resnet18 Feature Module 파라미터 requires grad = False => True 비활성화
for name, param in res_model.fc.named_parameters():
    print(name, param.requires_grad, end=' ====>    ')
    param.requires_grad = True
    print(param.requires_grad)

conv1.weight True ====>    False
bn1.weight True ====>    False
bn1.bias True ====>    False
layer1.0.conv1.weight True ====>    False
layer1.0.bn1.weight True ====>    False
layer1.0.bn1.bias True ====>    False
layer1.0.conv2.weight True ====>    False
layer1.0.bn2.weight True ====>    False
layer1.0.bn2.bias True ====>    False
layer1.1.conv1.weight True ====>    False
layer1.1.bn1.weight True ====>    False
layer1.1.bn1.bias True ====>    False
layer1.1.conv2.weight True ====>    False
layer1.1.bn2.weight True ====>    False
layer1.1.bn2.bias True ====>    False
layer2.0.conv1.weight True ====>    False
layer2.0.bn1.weight True ====>    False
layer2.0.bn1.bias True ====>    False
layer2.0.conv2.weight True ====>    False
layer2.0.bn2.weight True ====>    False
layer2.0.bn2.bias True ====>    False
layer2.0.downsample.0.weight True ====>    False
layer2.0.downsample.1.weight True ====>    False
layer2.0.downsample.1.bias True ====>    False
layer2.1.conv1.weight True ====>    False


In [8]:
for idx, module in enumerate(res_model.modules()):
    print(f"idx: {idx}, module: {module}")

idx: 0, module: ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(

In [9]:
# 학습준비 
optimizer = optim.Adam(res_model.fc.parameters())   # fc만 줘야함!!
loss_fn = nn.CrossEntropyLoss()   # 손실함수 정의
EPOCHS = 3